In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.8 MB/s eta 0:00:00


In [2]:
import os
#OPENAI_API_KEY = replaced
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
import openai

In [6]:
prompt = input('Enter a string for sentiment analysis')
completion = openai.ChatCompletion.create(model = 'gpt-3.5-turbo',
                                          messages = [
                                              {"role":"system", "content" : "classify the user message as positive or negative sentiment"},
                                              {"role":"user", "content":prompt}

                                          ])
print("\n"+ completion.choices[0].message.content)

Enter a string for sentiment analysisI am a Super Saiyan

The sentiment of the user message is positive.


In [8]:
completion.usage

<OpenAIObject at 0x7f6384326d40> JSON: {
  "prompt_tokens": 26,
  "completion_tokens": 9,
  "total_tokens": 35
}

In [7]:
completion

<OpenAIObject chat.completion id=chatcmpl-7rAsJ376uEgfothvK1GiuJvagmRey at 0x7f63844b9850> JSON: {
  "id": "chatcmpl-7rAsJ376uEgfothvK1GiuJvagmRey",
  "object": "chat.completion",
  "created": 1692907543,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The sentiment of the user message is positive."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 26,
    "completion_tokens": 9,
    "total_tokens": 35
  }
}

# Function calling

In [9]:
import openai
import json


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "22",
        "unit": unit,
        "forecast": ["sunny", "windy","supernatural phenomena", "alien invasion"],
    }
    return json.dumps(weather_info)


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())

{
  "id": "chatcmpl-7rAxaXfJmuCorccuzlvH1LxKWHfEB",
  "object": "chat.completion",
  "created": 1692907870,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston is 22 degrees Fahrenheit. It is sunny with some wind. Please be advised that the additional information provided about supernatural phenomena and alien invasion is fictional and not part of the weather report."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 42,
    "total_tokens": 124
  }
}
